### Social Network Graph
The first part of this project is modeling the social network over which we will simulate different events, namely parties, decay of relationships, falling outs, and meeting people.

We determined that the components of the graph are
* People: Each person is represented by a node
* Frendship / connection: Two connected people have an edge between them. The frequency of there being a connected is modeled off of Dunbar's number [(source)](https://www.bbc.com/future/article/20191001-dunbars-number-why-we-can-only-maintain-150-relationships).
* Friendship level: The friendship level of these two people are represented as the weight of the edge. The specific friendship level is chosen using Dunbar's number [(source)](https://www.bbc.com/future/article/20191001-dunbars-number-why-we-can-only-maintain-150-relationships). The specific break downs are:
    * Enemies (??): -1 
    * People you can recognize (1500): $\frac{0}{2215} \le x < \frac{1500}{2215}$.For most of our calculations, we will considert this range essentially 0.
    * Acquaintances (500): $\frac{1500}{2215} \le x < \frac{2000}{2215}$
    * Meaningful contacts (150): $\frac{2000}{2215} \le x < \frac{2150}{2215}$
    * Friends (50): $\frac{2150}{2215} \le x < \frac{2200}{2215}$
    * Good Friends (15): $\frac{2200}{2215} \le x < \frac{2215}{2215}$

In [583]:
import numpy as np
import heapq as pq
import math

In [54]:
def generate_graph(n: int, p:float):
    # generate the graph parameters
    connections = np.random.uniform(low=0, high=1, size=((n+1)*(n)//2)) > p
    weights = np.random.uniform(low=0, high=1, size=((n+1)*(n)//2))

    # fill out actual graph
    graph = np.zeros((n, n))
    ui = (np.triu_indices(n)) # indices of the upper triangular matrix
    graph[ui] = weights
    graph[ui] = graph[ui] * connections# np.ma.masked_array(graph[ui], mask=connections) 

    # transpose edges for undirected graph
    graph = graph + np.transpose(graph[:, :])

    x = np.arange(n)

    graph[x, x] = 0 
    
    return graph

In [169]:
graph = generate_graph(4000, 1-2215/4000)
graph

array([[0.        , 0.        , 0.47022782, ..., 0.        , 0.06182299,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.89192369,
        0.83592907],
       [0.47022782, 0.        , 0.        , ..., 0.        , 0.78355933,
        0.21922021],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.06182299, 0.89192369, 0.78355933, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.83592907, 0.21922021, ..., 0.        , 0.        ,
        0.        ]], shape=(4000, 4000))

In [ ]:
# This function generates the social connections (direct and indirect) between some given person and all other people. In the instance where this person and another person
# are directly connected, the 'social connection' value will likely be equivalent to their current friendship level. Otherwise, indirect connects can have a social connection
# with the given person if they are typically w/in one degree, with a high level of friendship to the intermediary (some friends, and good friends. Occurs about 1% of the time)
# AND the intermediary has a high degree of friendship to the original person. A friend of a friend, so to speak.

def generate_social_connections(graph, person):
    # Build the initial social connections based off the person's direct network
    connectedness = graph[person].copy()
    
    # Note the blacklisted people
    black_list = np.where(connectedness == -1)[0]

    # Create a filter that removes blacklisted people
    mask = np.zeros(connectedness.size, dtype=bool)
    mask[black_list] = True
    mask[person] = True
    mask = np.vstack((mask, mask))
    

    # Apply the filter to connectedness, giving possible guests
    possible = np.ma.array(np.vstack((connectedness, np.array([1]*connectedness.shape[0]))), mask=mask)

        #print("step 1", possible) #testing
    
    # Continue below operations until the social connection value is arbitrarily low (within the lowest catgory)
    while (possible[0] >= 1500/2215).sum() > 0:

            #print("entered loop!") #testing
        
        # Choose the next highest social connection
        next_guest = np.argmax(possible[0])
        guest_distance = possible[1, next_guest]
        connectedness[next_guest] = possible[0, next_guest]

            #print("next guest:", next_guest, "distance:", guest_distance) #testing

        # Update filter to include the next added guest since we do not want to update this value.
        mask[:, next_guest] = True
        possible = np.ma.array(possible, mask=mask)
            #print("masked off next guest", possible) #testing

        
        # Improve social connections if a person is indirectly connected to the person
        updater = possible[0] < connectedness[next_guest] * graph[next_guest] / (guest_distance + 1)**(1/1.83)
            #print("updater", updater) # testing
        possible[1, updater] = guest_distance + 1
        possible[0, updater] = (connectedness[next_guest] * graph[next_guest] / (possible[1])**(1/1.83))[updater]

            #print("updated possible", possible) #testing
    
    return connectedness

In [566]:
test_graph = generate_graph(4000, 1-2215/4000)
connections = generate_social_connections(test_graph, 4)

print("Direct social connections vs. Total Social Connections")
print("Total edge weight:",sum(test_graph[4]), sum(connections))
print("Number of important edges(non-arbitrary connection):", sum(test_graph[4] > 1500/2215), sum(connections > 1500/2215))
print("Total number of edges:", sum(test_graph[4] > 0), sum(connections > 0))

print("\nGranular breakdown of direct and total social connections")
print("Number of acquaintances+:", sum((test_graph[4]) > 1500/2215),sum(connections > 1500/2215))
print("Number of meaningful contacts+:", sum(test_graph[4] > 2000/2215),sum(connections > 2000/2215))
print("Number of friends+:", sum(test_graph[4]>2150/2215), sum(connections > 2150/2215))
print("Number of good friends+:", sum(test_graph[4]>2200/2215),sum(connections > 2200/2215))

Direct social connections vs. Total Social Connections
Total edge weight: 1094.3167086948886 1243.2467921840896
Number of important edges(non-arbitrary connection): 719 1009
Total number of edges: 2206 2354

Granular breakdown of direct and total social connections
Number of acquaintances+: 719 1009
Number of meaningful contacts+: 210 210
Number of friends+: 67 67
Number of good friends+: 17 17


In [586]:
(1- ((1500/2215) * (2)**(1/1.83))) #Rough percentage of occurence

0.010957864640372073